In [ ]:
import pandas as pd

df = pd.read_csv('../data/clean/wandb_export.csv')

df.rename(columns={
    'bins' : 'Task',
    'gat' : 'Model Type',
    'graph_num' : 'Feature Set',
    'hidden_c' : 'Hidden Channels',
    'num_heads' : 'Attention Heads',
    'val_acc (Max)' : 'Validation Accuracy*',
}, inplace=True)

def task_mapper (task):
    if task == '[3000]':
        return 'Binary Classification'
    elif task == '[400,800,1300,2100,3000,3700,4700,7020,9660]':
        return 'Multi-class Classification'
    elif task == 'regression':
        return 'Regression'
    
model_mapper = {
    'MLP' : 'MLP',
    'false' : 'GCN',
    'true' : 'GAT',
}

model_mapper_rev = {v: k for k, v in model_mapper.items()}
task_mapper_rev = {
    'Binary Classification': '[3000]',
    'Multi-class Classification': '[400,800,1300,2100,3000,3700,4700,7020,9660]',
    'Regression': 'regression',
}

df['Task'] = df['Task'].apply(task_mapper)
df['Model Type'] = df['Model Type'].map(model_mapper)
df['Validation Accuracy*'] = df['Validation Accuracy*'].round(3)

In [ ]:
df.columns

In [ ]:
import sys
sys.path.append("../scripts")

import os, torch
from sklearn.model_selection import train_test_split
import pickle
import torch_geometric.transforms as T
import numpy as np
from torch_geometric.nn.models import Node2Vec
from torch_geometric.data import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
GCNConv._orig_propagate = GCNConv.propagate
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch_geometric.explain import GNNExplainer, Explainer
from models import *
from tg_functions import *
from bike_functions import *


def test(model, data, criterion, device, bins):
    model.eval()
    data.x = data.x.to(device)
    data.edge_index = data.edge_index.to(device)
    data.test_mask = data.test_mask.to(device)
    data.y = data.y.to(device)
    mask = data.test_mask.squeeze() & (data.y > 0).squeeze()
    out = model(data.x, data.edge_index)
    #target = torch.bucketize(data.y[mask], bins).squeeze()
    if bins == 'regression':
        target = data.y[mask].squeeze()
        loss = criterion(out[mask].squeeze(), target)
        accuracy = r2_score(target.detach().cpu().numpy(), out[mask].detach().cpu().numpy())
    else:
        target = torch.bucketize(data.y[mask], bins).squeeze()
        loss = criterion(out[mask], target.long())  # Ensure target is 1D and long
        correct_preds = out[mask].argmax(dim=1)
        correct = (correct_preds == target).sum()
        accuracy = correct.item() / mask.sum().item()
    return accuracy, out, loss.item()

dropout_p = 0.5
use_gat = True

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}", flush = True)
else:
    device = torch.device('cpu')
    print("Using CPU", flush = True)

test_accuracies = []

for model_name, graph_num, model_type, hidden_channels, num_heads, bins in zip(df['Name'], df['Feature Set'], df['Model Type'], df['Hidden Channels'], df['Attention Heads'], df['Task']):
    print(f"Processing model: {model_name}, Feature Set: {graph_num}, Model Type: {model_type}, Hidden Channels: {hidden_channels}, Attention Heads: {num_heads}, Task: {bins}", flush=True)
    if bins == 'Regression':
        bins = bins.lower()    
    if bins != 'regression':
        bins = task_mapper_rev[bins]
        bins = [int(i) for i in bins.replace('[', '').replace(']', '').split(',')]
        bins = torch.tensor(bins, device='cuda' if torch.cuda.is_available() else 'cpu')

    with open(f'../data/graphs/{graph_num}/linegraph_tg.pkl', 'rb') as f:
        data = pickle.load(f)

    data.edge_index = data.edge_index.contiguous()
    data.x = data.x.contiguous()
    data.y = data.y.contiguous()
    data = stratified_split(data = data , random_seed = 100)

    torch.manual_seed(100)
        
    if model_type == 'MLP':
        model = MLP(data = data,hidden_channels= hidden_channels, random_seed=100, num_layers=0, bins=bins).to(device)
    elif model_type == 'GCN':
        model = GCN(data = data, hidden_channels= hidden_channels, random_seed=100, num_layers=0, bins=bins).to(device)
    elif model_type == 'GAT':
        model = GAT(data = data, hidden_channels= hidden_channels, num_heads=num_heads, random_seed=100, num_layers=0, bins=bins).to(device)
    # Load the pre-trained model
    model = torch.load(f'../data/graphs/{graph_num}/models/{model_name}.pt', map_location=device)
    model = model.to(device)
    try:
        model.load_state_dict(torch.load(f'../data/graphs/{graph_num}/models/{model_name}_best_accuracy.pt', map_location=device))
    except:
        model.load_state_dict(torch.load(f'../data/graphs/{graph_num}/models/{model_name}_best_loss.pt', map_location=device))

    model.eval()

    criterion = torch.nn.CrossEntropyLoss() if bins != 'regression' else torch.nn.MSELoss()

    accuracy, out, loss = test(model, data, criterion, device, bins)
    test_accuracies.append(accuracy)
    

In [ ]:
df['Test Accuracy'] = test_accuracies
df['Test Accuracy'] = df['Test Accuracy'].round(3)

In [ ]:
df

In [ ]:
df.to_latex('../data/clean/wandb_export.tex', index=False, float_format='%.3f',
            column_format='lcccccc',
            escape=False,
            caption='Results of the experiments conducted on the benchmark datasets. The validation accuracy is reported as the maximum value achieved during training. The task type indicates whether the task is binary classification, multi-class classification, or regression. Accuracy values are rounded to three decimal places. For classification tasks, the accuracy is computed as the average of the per-class accuracies. For regression tasks, the accuracy is computed as the R-squared score.',
            label='tab:wandb_export')